# Model Starter + Tensorflow + GEM Pool + MixedPrec

- Starter baseline for getting in contact with the competition structure
- Gem pooling helps to remove noise when pooling alongisde channel axis on the backbone model.
- Fast dataset piepeline to increase training efficiency.
- mixed precission to improve training speed and memory.
- Tips to improve: 
    - Increase img_size, better model, more augmentations, decrease lr along the epochs.
- Be careful to not exceed 9h of training!!

In [ ]:
#################################################################
# 0. Libraries

import pandas as pd
import numpy as np
import os

from tqdm import tqdm

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import models, layers, regularizers, metrics, losses, optimizers, constraints
from tensorflow.keras import mixed_precision
from tensorflow.keras.utils import Sequence

from sklearn.model_selection import train_test_split

np.random.seed(12)

#################################################################

In [ ]:
#################################################################
# 1. Global Variables & Paths

PATH = '../input/landmark-recognition-2021/'
PATH_MODELS = './03_Models/'

df_train = pd.read_csv(PATH + 'train.csv')
df_train['path'] = df_train['id'].apply(lambda x: PATH +  'train/' + '/'.join([c for c in x[:3]]) + '/' + x + '.jpg')

# Missing labels between 0-num_classes causes NAN for sparse_categorical_crossentropy.
dict_map_landmark = {l : i for i, l in enumerate(df_train.landmark_id.unique())}
dict_map_landmark_inv = {i : l for i, l in enumerate(df_train.landmark_id.unique())}
df_train['landmark_id_mapped'] = df_train['landmark_id'].map(dict_map_landmark)

IMG_SIZE = (320, 320, 3)
IMG_SIZE_CROP = (280, 280, 3)

if not os.path.exists(PATH_MODELS):
    os.mkdir(PATH_MODELS)
    
# Get test paths
test_filenames=[]
for dirname, _, filenames in tqdm(os.walk(PATH + 'test')):
    for filename in filenames:
        test_filenames.append(filename.split(".")[0])
        
df_test = pd.DataFrame({"id": test_filenames,"landmarks":""})
df_test['path'] = df_test['id'].apply(lambda x: PATH +  'test/' + '/'.join([c for c in x[:3]]) + '/' + x + '.jpg')

#################################################################

In [ ]:
#################################################################
# 2. Aux functions

def plot_examples_train(landmark_id=1):
    fig, axs = plt.subplots(1, 5, figsize=(26, 12))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    len_ = len(df_train[df_train['landmark_id']==landmark_id])
    for i in range(5):
        if i>=len_:
            break
        idx = df_train[df_train['landmark_id']==landmark_id].index[i]
        path_ = df_train.loc[idx, 'path']
        file_bytes = tf.io.read_file(path_)
        img = tf.image.decode_jpeg(file_bytes, channels=3)
        img = tf.image.resize(img, size=(IMG_SIZE[0], IMG_SIZE[1]))
        axs[i].imshow(img / 255.)
        axs[i].set_title('landmark_id: '+str(landmark_id) + str(i) + '/' + str(len_))
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        
def plot_img_path(path, figsize=(6, 6)):
    file_bytes = tf.io.read_file(path)
    img = tf.image.decode_jpeg(file_bytes, channels=3)
    img = tf.image.resize(img, size=(IMG_SIZE[0], IMG_SIZE[1]))
    plt.figure(figsize=figsize)
    plt.title('Path: '+path)
    plt.imshow(img / 255.)
    plt.show()

#################################################################

In [ ]:
#################################################################
# 3. Some plots

# Train

unique_landmarks_train = df_train.landmark_id.unique()

landmark_ = np.random.choice(unique_landmarks_train, 1)[0]
plot_examples_train(landmark_)

landmark_ = np.random.choice(unique_landmarks_train, 1)[0]
plot_examples_train(landmark_)

landmark_ = np.random.choice(unique_landmarks_train, 1)[0]
plot_examples_train(landmark_)

# Test

unique_paths_test = df_test.path.unique()
path_test = np.random.choice(unique_paths_test, 1)[0]
plot_img_path(path_test)

path_test = np.random.choice(unique_paths_test, 1)[0]
plot_img_path(path_test)

path_test = np.random.choice(unique_paths_test, 1)[0]
plot_img_path(path_test)

#################################################################

In [ ]:
#################################################################
# 4. DataLoader


def buildLoader(labels=True):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        img = tf.image.decode_jpeg(file_bytes, channels=3)
        img = tf.cast(img, tf.float32) / 255.
        return img
    
    def decodeWithLabels(path, label):
        img = decode(path)
        return img, label
    
    if labels:
        return decodeWithLabels
    else:
        return decode


def preprocessImage(img, label):
    img = tf.image.resize(img, size=(IMG_SIZE[0], IMG_SIZE[1]))
    img = tf.image.random_crop(img, size=(IMG_SIZE_CROP[0], IMG_SIZE_CROP[1], 3))
    label = tf.expand_dims(label, -1)
    img = tf.cast(img, tf.float32)
    label = tf.cast(label, tf.int32)
    return img, label


def set_shapes(img, label):
    img.set_shape(IMG_SIZE_CROP)
    if label is not None:
        label.set_shape([1])
    return img, label


def buildAugmentations():
    def applyAugmentations(image, label):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        return image, label
    return applyAugmentations


def build_dataset(paths, labels=None, bsize=32, shuffle=True, augmentations=True):
    aug_builder = buildAugmentations()
    loader = buildLoader(False if labels is None else True) 
    AUTO = tf.data.AUTOTUNE
    slices = (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    if shuffle:
        dset = dset.shuffle(len(labels))
        
    dset = dset.map(loader, num_parallel_calls=AUTO).prefetch(AUTO)
    dset = dset.map(preprocessImage, num_parallel_calls=AUTO)
    if augmentations:
        dset = dset.map(aug_builder, num_parallel_calls=AUTO)
    dset = dset.map(set_shapes, num_parallel_calls=AUTO)
    dset = dset.batch(bsize, drop_remainder=True).prefetch(AUTO)
    return dset   

#################################################################

In [ ]:
df_tmp = df_train.sample(100)

list_paths = list(df_tmp['path'])
list_labels =list(df_tmp['landmark_id_mapped'])
dataset_train = build_dataset(list_paths, labels=list_labels, bsize=64, shuffle=False)
for batch in tqdm(dataset_train):
    data, target = batch
    break

data, target = batch

idx = 0
print(data.shape, target.shape)
print(target[idx])
plt.title(f"TARGET: {target[idx].numpy()}", fontsize=16)
plt.imshow(data[idx]);plt.show();

idx = 12
print(data.shape, target.shape)
print(target[idx])
plt.title(f"TARGET: {target[idx].numpy()}", fontsize=16)
plt.imshow(data[idx]);plt.show();

idx = 25
print(data.shape, target.shape)
print(target[idx])
plt.title(f"TARGET: {target[idx].numpy()}", fontsize=16)
plt.imshow(data[idx]);plt.show();

In [ ]:
#################################################################
# 5. Model

class GeMPoolingLayer(tf.keras.layers.Layer):
    def __init__(self, p=1., train_p=False, mixed_prec=True):
        super().__init__()
        if train_p:
            if mixed_prec:
                self.p = tf.Variable(p, dtype=tf.float16)
            else:
                self.p = tf.Variable(p, dtype=tf.float32)
        else:
            self.p = p
        self.eps = 1e-7

    def call(self, inputs: tf.Tensor, **kwargs):
        inputs = tf.clip_by_value(inputs, clip_value_min=self.eps, clip_value_max=tf.reduce_max(inputs))
        inputs = tf.pow(inputs, self.p)
        inputs = tf.reduce_mean(inputs, axis=[1, 2], keepdims=False)
        inputs = tf.pow(inputs, 1./self.p)
        return inputs

    
class Model(models.Model):
    def __init__(self, num_classes, mixed_prec=True):
        super(Model, self).__init__()
        self.backbone_model = tf.keras.applications.InceptionV3(input_shape=(IMG_SIZE_CROP[0], IMG_SIZE_CROP[1], 3),
                                                                include_top=False, weights='../input/inception-v3-weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
        self.backbone_model.trainable = True
        self.gem_pool = GeMPoolingLayer(mixed_prec)
        self.fc_linear = layers.Dense(512, activation='linear')
        self.fc_bn = layers.BatchNormalization()
        
        self.dense_out = layers.Dense(num_classes, activation='linear')
        self.dense_mp = layers.Activation('softmax', dtype='float32')
        
    def call(self, img_input, training):
        x = self.backbone_model(img_input, training)
        x = self.gem_pool(x)
        x = self.fc_linear(x)
        x = self.fc_bn(x)
        x = tf.nn.relu(x)
        
        logits = self.dense_mp(self.dense_out(x))
        return logits
    
    
def getModel(num_classes, mixed_prec):
    return Model(num_classes, mixed_prec)

#################################################################

In [ ]:
#################################################################
# 6. Training

tf.keras.backend.clear_session()
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

batch_size = 38
epochs = 1
model_base_name = 'model_v0.1'
num_classes = len(df_train['landmark_id'].unique())
    
tup_splitted = train_test_split(df_train['path'].values, df_train['landmark_id_mapped'].values, 
                                                    test_size=0.3, random_state=12, stratify=df_train['landmark_id_mapped'].values)

list_paths_train, list_paths_val, list_labels_train, list_labels_val = tup_splitted

print(f'Num paths train: {len(list_paths_train)}, Num paths val: {len(list_paths_val)}')

train_data_generator = build_dataset(list_paths_train, labels=list_labels_train, bsize=batch_size, augmentations=True, shuffle=True)
val_data_generator = build_dataset(list_paths_val, labels=list_labels_val, bsize=batch_size, augmentations=False, shuffle=False)

# Model
model = getModel(num_classes, mixed_prec=True)

learning_rate = 1e-4
model.compile(optimizer=mixed_precision.LossScaleOptimizer(optimizers.Adam(learning_rate)),
              loss=losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=[metrics.SparseCategoricalAccuracy(name='acc'), 
                       metrics.SparseTopKCategoricalAccuracy(k=5, name='acc_top5')])

history = model.fit(train_data_generator,
                    validation_data=val_data_generator,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

if not os.path.exists(PATH_MODELS + model_base_name):
    os.mkdir(PATH_MODELS + model_base_name)

model.save(f'{PATH_MODELS + model_base_name}/{model_base_name}', include_optimizer=False)
model.save_weights(f'{PATH_MODELS + model_base_name}/{model_base_name}_weights', save_format='tf')

#################################################################

In [ ]:
#################################################################
# 3. Inference and submission

bsize = 32
model_load_name = 'model_v0.1'
model = models.load_model(PATH_MODELS + model_load_name + '/' + model_load_name)

list_paths_test = list(df_test['path'].values) 
test_data_generator = build_dataset(list_paths_test, labels=np.zeros(len(list_paths_test)), bsize=bsize, augmentations=False, shuffle=False)

list_id, list_pred = [], []
for index, batch in enumerate(tqdm(test_data_generator)):
    y_pred = model(batch[0], training=False).numpy()
    pred_landmark = np.argmax(y_pred, 1) 
    score_landmark = [y_pred[i, pred_landmark[i]].round(2) for i in range(y_pred.shape[0])]
    category = [dict_map_landmark_inv[x] for x in pred_landmark]
    for i in range(y_pred.shape[0]):
        df_test.loc[index+i, 'landmarks'] = str(category[i]) + ' ' + str(score_landmark[i])

df_test[['id', 'landmarks']].to_csv('./submission.csv', index=False)

#################################################################